In [4]:
def utm_finder(raster):
    """
    Find the UTM EPSG code of a non-utm projected raster.
    Arguments:  
    raster: input raster path
    Returns:
    UTM EPSG code of the input raster
    """
    with rasterio.open(raster) as dataset:       
        bbox  = dataset.bounds
        bbox_wgs84 = rasterio.warp.transform_bounds(dataset.crs,'EPSG:4326', bbox[0],bbox[1],bbox[2],bbox[3])

        utm_crs_list = query_utm_crs_info(     
            datum_name='WGS 84',
            area_of_interest=AreaOfInterest(
            west_lon_degree=bbox_wgs84[0],
            south_lat_degree=bbox_wgs84[1],
            east_lon_degree=bbox_wgs84[2],
            north_lat_degree=bbox_wgs84[3],),) 

        utm_crs = '{}:{}'.format(utm_crs_list[0].auth_name,utm_crs_list[0].code) 
    return utm_crs

In [5]:
def resample(raster,pixel_size,output_dir,output_name):
    """
    Resample a raster to a given pixel size.
    Arguments:  
        raster: input raster path
        pixel_size: resolution in meters
        output_dir: output folder path
        output_name: raster name with extension
    Returns:
        Resampled raster   
    """
    with rasterio.open(raster) as dataset:
        kwargs = dataset.meta.copy()
        # resample data to target shape
        upscale_factor = dataset.transform[0]/pixel_size
        
        data = dataset.read(out_shape=(dataset.count,int(dataset.height * upscale_factor),int(dataset.width * upscale_factor)),resampling=Resampling.bilinear)
        # scale image transform
        transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2]))
        
        # writing file to disk 
        kwargs.update({'transform': transform,
                       'width': data.shape[2],
                       'height': data.shape[1]})
        
        with rasterio.open(os.path.join(output_dir,output_name), 'w', **kwargs) as dst:
            dst.write(data[0], 1)
            
        band = None
            
        

In [6]:
def calibration(raster,cosys,output_dir,output_name):
    """
    Raster calibration,speckle reduction, and reprojection.
    Arguments:  
        raster: input raster path
        cosys: epsg code of the desired coordinate system
        output_dir: output folder path
        output_name: raster name with extension
    Returns:
        Calibrated raster in desired projection     
    """
    with rasterio.open(raster) as dataset:
        # tranformation calculation for UTM reprojection
        transform, width, height = calculate_default_transform(
            dataset.crs,
            cosys,
            dataset.width,
            dataset.height,
            *dataset.bounds)
        
        kwargs = dataset.meta.copy()
        kwargs.update({
            'crs': cosys,
            'transform': transform,
            'width': width,
            'height': height,
            'compress': 'lzw',
            'dtype':'float32',
            'nodata':np.nan})
        
        # Calibration
        band = dataset.read(1)
        band = np.float32(np.where(band == 0, np.nan, band))
        band_calib = 20*np.log10(band)-83

        # Filtering
        med =  medfilt2d(band_calib, kernel_size=3)
                  
        with rasterio.open(os.path.join(output_dir,output_name), "w", **kwargs) as dst:           
            for i in range(1, dataset.count + 1):
                reproject(
                    source=med,
                    destination=rasterio.band(dst, i),
                    src_transform=dataset.transform,
                    src_crs=dataset.crs,
                    dst_transform=transform,
                    dst_crs=cosys,
                    resampling=Resampling.nearest)
            
        med = None
        band = None
        band_calib = None
                     

In [1]:
def tile512(raster,output_dir,pol_suffix):    
    """
    Create raster tiles of 512 by 512
    Arguments:
        raster: input raster path
        output_dir: output folder path
        pol_suffix: string of suffix based on polarizaiton eg -> hh or hv -> T0101_hh.tif     
    Returns:
        512 by 512 raster tiles    
    """
    
    with rasterio.open(raster) as src: 
        width = src.width 
        height = src.height 
        res =  src.transform[0]
        UL_c,UL_r = src.transform * (0, 0)   
        band = src.read(1)       
        kwargs = src.meta.copy()
                   
        for col in range(int(np.round_(width/512))):           
            for row in range(int(np.round_(height/512))):
                tf_tile = Affine(res, 0.0,UL_c+(512*res*col),0.0,-1*(res),UL_r+(512*(-1*(res))*row))               
                tile_name = "T{:02d}".format(row+1)+"{:02d}_{pol}.tif".format(col+1,pol=pol_suffix)               
                np_arr = np.zeros((512, 512), dtype='float32')                
                tile_band = band[512*row:512*row+512,512*col:512*col+512]              
                np_arr[:tile_band.shape[0], :tile_band.shape[1]] = tile_band
                
                kwargs.update({
                    'transform': tf_tile,
                    'width': 512,
                    'height': 512,
                    'compress': 'lzw',
                    'dtype':'float32',
                    'nodata':np.nan})
                                
                with rasterio.open(os.path.join(output_dir,tile_name), "w", **kwargs) as dst:
                    dst.write(np_arr,1)

        np_arr = None 
        tile_band = None
        band = None
        

In [ ]:
def restgee_data(input_tile,ee_img,ee_img_band,output_dir):
    """
    Download GEE objectecs correspond to raster tile extent using EE REST API(restee package)
    Arguments:  
        input_tile: Input calibrated raster tile 
        ee_img: GEE object
        ee_img_band: GEE object band name
        output_dirraster: output directory path    
    Returns:
        GEE objectecs as tiles correspoding to calibrated raster tiles
    """
    with rasterio.open(input_tile) as src:
        kwargs = src.meta.copy()
        tile_tranform = src.transform
        tile_bounds = src.bounds
        tile_crs = src.crs     
        gee_domain = ree.Domain((tile_bounds[0],
                                 tile_bounds[1],
                                 tile_bounds[2],
                                 tile_bounds[3]),
                                resolution= src.transform[0],
                                crs =str(tile_crs))
        band_utm = np.int32(ree.img_to_ndarray(ee_session, gee_domain, image=ee_img, bands=ee_img_band))      
        dst_tranform = rasterio.transform.from_bounds(tile_bounds.left, 
                                                      tile_bounds.bottom,
                                                      tile_bounds.right, 
                                                      tile_bounds.top, 
                                                      band_utm.shape[1],
                                                      band_utm.shape[0])      
        out_file_path = os.path.join(output_dir,'{}_{}.tif'.format(input_tile.split('/')[-1].split('.tif')[0].split('_')[0],ee_img_band))
        with rasterio.open(out_file_path, "w", **kwargs) as dst:
            reproject(
                source=band_utm,
                destination=rasterio.band(dst,1),
                src_transform=dst_tranform,
                src_crs=tile_crs,
                dst_transform=tile_tranform,
                dst_crs=tile_crs,
                resampling=Resampling.nearest)   

In [ ]:
def pwater_slope_masking(raster,pwater_mask,slope_mask,output_dir):
    """
    Masking out slope(>5) and permenent water areas from calibrated raster tiles
    Arguments:  
        raster: input raster chip path
        pwater_mask: input permanent water mask chip path
        slope_mask: input slope mask chip path
        output_dir: output directory path
    Returns:
        High slope and permanent water masked raster    
    """
    
    ras = rasterio.open(raster)
    kwargs = ras.meta.copy()
    ras_band = ras.read(1)    
    pwater_mask = rasterio.open(pwater_mask) 
    slope_mask = rasterio.open(slope_mask) 
    
    pwater_mask_band = pwater_mask.read(1)
    pwater_mask_band[pwater_mask_band==1.0]=np.nan
    pwater_mask_band[pwater_mask_band==0.0]=1.0      
    slope_mask_band = slope_mask.read(1)
    slope_mask_band[slope_mask_band==0]=np.nan   
    masked_arr = ras_band*pwater_mask_band*slope_mask_band 
    outfile_name = os.path.join(output_dir,os.path.basename(raster).split('.tif')[0]+'_masked.tif')
    
    with rasterio.open(outfile_name, 'w', **kwargs) as dst:
        dst.write(masked_arr, 1)   
        
    ras_band = None 
    slope_mask_band = None
    pwater_mask_band = None
    masked_arr = None
    
    ras.close()
    pwater_mask.close()
    slope_mask.close()

In [ ]:
def iqr_mean(values_list):
    '''
    Removes outliers from a list and calcuate the mean value of the filtered list
    Arguments:  
        values_list: list to remove outliers and then calculate mean
    Returns:
        mean of the filtered list
    '''
    sorted_list = np.array(sorted(values_list))
    q1, q3= np.percentile(sorted_list,[25,75])
    iqr = q3 - q1
    lower_bound = q1 -(1.5 * iqr)
    upper_bound = q3 +(1.5 * iqr)
    filtered_list = sorted_list [ (sorted_list >=lower_bound) * (sorted_list <= upper_bound)]
    return(round(np.mean(filtered_list),2))

In [1]:
def bin_threshoding(raster_list,output_dir,thresh_val):
    '''
    Apply threshold for a given raster list
    Arguments:  
        raster_list: input raster list
        output_dir: output directory
        thresh_val: Threshold value for binary classification
    Returns:
        Binary rasters based on the defined threshold value
    '''
    with tqdm(total=len(raster_list),position=0, leave=True, desc="Binary thresholding progress") as pbar:
        for raster in raster_list:
            with rasterio.open(raster) as dataset:
                kwargs = dataset.meta.copy()
                outfile_bin_filt_path = os.path.join(output_dir,
                                                     os.path.basename(raster).split('.tif')[0]+'_bin_filt.tif')
                band = dataset.read(1)
                band_bin = (band < thresh_val)
                #filtering using median filter
                band_bin_filt =  medfilt2d(np.float32(band_bin), kernel_size=3) 
                
                with rasterio.open(outfile_bin_filt_path, 'w', **kwargs) as dst:
                    dst.write(band_bin_filt, 1) 
            pbar.update(n=1)

In [ ]:
def mosaic_rastiles(raster_list,output_dir,output_name):
    '''
    Mosaic raster tiles in to a one raster
    Arguments:  
        raster_list: input raster list
        output_dir: output directory
        output_name: Output file name
    Returns:
        Mosaiced raster
    '''
    # Getting the projection from a raster in raster list
    epsg_code = raster_list[0]
    src_sample = rasterio.open(raster_list[0])
    mosaic, out_trans = merge(raster_list)
    out_meta = {'driver': "GTiff",
                'height': mosaic.shape[1],
                'width': mosaic.shape[2],
                'transform': out_trans,
                'crs': src_sample.crs,
                'count':1,
                'compress': 'lzw',
                'dtype':'float32',
                'nodata':np.nan}
    mosaic = mosaic[0]
    mosaic[mosaic==0.0]=np.nan
    src_sample.close()
             
    with rasterio.open(os.path.join(output_dir,output_name), "w", **out_meta) as dest:
        dest.write(mosaic,1)